In [1]:
import SSD_

In [2]:
print("pass")

pass


In [3]:
# Trained From NewEMEnhanceWithTrans_1114_G_340
# Trained From NewEMEnhanceWithTrans_1126_G_80
import torch
import torch.nn as nn
import math
import torch.nn.functional as F
from torch.utils.checkpoint import checkpoint
from torch.nn import init
import functools
from torch.optim import lr_scheduler

# Actually we don't use Single Dense Layer in this model.
class SingleDenseLayer(nn.Module):
	def __init__(self, input_dim=3, growthRate=12):
		super(SingleDenseLayer, self).__init__()
		self.activation = nn.ELU(inplace=True)
		self.bn = nn.BatchNorm2d(num_features=input_dim)
		self.conv1 = nn.Conv2d(in_channels=input_dim,out_channels=growthRate,kernel_size=3,stride=1,padding=1,bias=False)
	def forward(self, x):
		output = self.conv1(self.activation(self.bn(x)))
		output = torch.cat((x,output),1)
		return output

# Each Dense Block in our model is made of 5 Bottle Neck Layers.
class BottleNeckLayer(nn.Module):
	def __init__(self, input_dim=3, growthRate=12):
		super(BottleNeckLayer, self).__init__()
		self.activation = nn.ReLU(inplace=True)
		# self.activation = nn.ELU(inplace=True)
		self.bn = nn.BatchNorm2d(num_features=int(input_dim))
		self.conv1 = nn.Conv2d(in_channels=input_dim,out_channels=4*growthRate,kernel_size=1,stride=1,padding=0,bias=False)
		self.conv2 = nn.Conv2d(in_channels=4*growthRate,out_channels=growthRate,kernel_size=3,stride=1,padding=1,bias=False)
	def forward(self, x):
		output = self.conv1(self.activation(self.bn(x)))
		output = self.conv2(output)
		output = torch.cat((x,output),1)
		return output

# The Transition Layer connects 2 Dense block.
class TransitionLayer(nn.Module):
	def __init__(self, input_dim, output_dim, is_pool=False):
		super(TransitionLayer, self).__init__()
		self.activation = nn.ReLU(inplace=True)
		self.bn = nn.BatchNorm2d(num_features=input_dim)
		self.conv = nn.Conv2d(in_channels=input_dim,out_channels=output_dim,kernel_size=1,stride=1,padding=0,bias=False)
		self.pool = nn.AvgPool2d(kernel_size=2,stride=2)
		self.is_pool = is_pool
	def forward(self, x):
		result = self.conv(self.activation(self.bn(x)))
		if(self.is_pool):
			result = self.pool(result)
		return result

class DenseBlock(nn.Module):
	def __init__(self, input_dim=3, growthRate=12, num_dense_layers=5):
		super(DenseBlock, self).__init__()
		
		self.im = input_dim
		layers = []
		for i in range(num_dense_layers):
			layers.append(BottleNeckLayer(input_dim+i*growthRate,growthRate))
		self.dense = nn.Sequential(*layers)
	
	def forward(self, x):
		return self.dense(x)

# ****** Define Unet ******
class DoubleConv(nn.Module):
    """(convolution => [BN] => ReLU) * 2"""

    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)


class Down(nn.Module):
    """Downscaling with maxpool then double conv"""

    def __init__(self, in_channels, out_channels):
        super(Down, self).__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)


class Up(nn.Module):
    """Upscaling then double conv"""

    def __init__(self, in_channels, out_channels, bilinear=True):
        super(Up, self).__init__()

        # if bilinear, use the normal convolutions to reduce the number of channels
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        else:
            self.up = nn.ConvTranspose2d(in_channels // 2, in_channels // 2, kernel_size=2, stride=2)

        self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        # input is CHW
        diffY = torch.tensor([x2.size()[2] - x1.size()[2]])
        diffX = torch.tensor([x2.size()[3] - x1.size()[3]])

        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)


class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)
		
class UNet(nn.Module):
    def __init__(self, n_channels, n_classes, bilinear=True):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear

        self.inc = DoubleConv(n_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        self.down4 = Down(512, 512)
        self.up1 = Up(1024, 256, bilinear)
        self.up2 = Up(512, 128, bilinear)
        self.up3 = Up(256, 64, bilinear)
        self.up4 = Up(128, 64, bilinear)
        self.outc = OutConv(64, n_classes)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        return logits

# ****** Define FCN-8 ******

class FCN8(nn.Module):

    def __init__(self, in_channel=3, numclass=3):
        super(FCN8, self).__init__()
        self.conv11 = nn.Conv2d(in_channels=in_channel, out_channels=64, kernel_size=3, padding=1)
        self.conv12 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv21 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.conv22 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv31 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1)
        self.conv32 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)
        self.conv33 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv41 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1)
        self.conv42 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)
        self.conv43 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2, padding=1)

        self.conv51 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)
        self.conv52 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)
        self.conv53 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1)
        self.pool5 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv6 = nn.Conv2d(in_channels=512, out_channels=4096, kernel_size=7, padding=3)
        self.dropout1 = nn.Dropout(0.85)

        self.conv7 = nn.Conv2d(in_channels=4096, out_channels=4096, kernel_size=1)
        self.dropout2 = nn.Dropout(0.85)

        self.conv8 = nn.Conv2d(in_channels=4096, out_channels=numclass, kernel_size=1)

        self.tranconv1 = nn.ConvTranspose2d(in_channels=numclass, out_channels=512, kernel_size=4, stride=2, padding=1)

        self.tranconv2 = nn.ConvTranspose2d(in_channels=512, out_channels=256, kernel_size=4, stride=2, padding=2, output_padding=1)

        self.tranconv3 = nn.ConvTranspose2d(in_channels=256, out_channels=numclass, kernel_size=16, stride=8, padding=4)

    def forward(self, x):
        x = self.conv12(self.conv11(x))
        x = self.pool1(x)
        x = self.pool2(self.conv22(self.conv21(x)))
        x1 = self.pool3(self.conv33(self.conv32(self.conv31(x))))
        x2 = self.pool4(self.conv43(self.conv42(self.conv41(x1))))
        x = self.pool5(self.conv53(self.conv52(self.conv51(x2))))
        x = self.dropout1(self.conv6(x))
        x = self.dropout2(self.conv7(x))
        x = self.conv8(x)
        x = self.tranconv1(x)
        x = x2 + x
        x = self.tranconv2(x)
        x = x1 + x
        x = self.tranconv3(x)
        return x


class Downsample(nn.Module):
	def __init__(self, in_channels, out_channels, scale):
		super(Downsample, self).__init__()
		layers = []
		num_channels = in_channels
		for i in range(int(math.log(scale, 2))-1):
			layers.append(Down(num_channels, num_channels*2))
			num_channels = num_channels * 2
		layers.append(Down(num_channels, out_channels))
		self.downsample = nn.Sequential(*layers)
		

	def forward(self, x):
		return self.downsample(x)
class Upsample(nn.Module):
	def __init__(self, in_channels, out_channels, scale):
		super(Upsample, self).__init__()
		layers = []
		num_channels = in_channels
		for i in range(int(math.log(scale, 2))-1):
			layers.append(nn.ConvTranspose2d(in_channels=num_channels, out_channels=num_channels//2, kernel_size=4, stride=2, padding=1))
			num_channels = num_channels // 2
		layers.append(nn.ConvTranspose2d(in_channels=num_channels, out_channels=out_channels, kernel_size=4, stride=2, padding=1))
		self.upsample = nn.Sequential(*layers)
		

	def forward(self, x):
		return self.upsample(x)

class PALayer(nn.Module):
    def __init__(self, channel):
        super(PALayer, self).__init__()
        self.activation = nn.ReLU(inplace=True)
        self.sigmoid = nn.Sigmoid()
        self.conv00 = nn.Conv2d(in_channels=channel, out_channels=channel, kernel_size=3, padding=1, bias=True)
        self.conv01 = nn.Conv2d(in_channels=channel, out_channels=channel, kernel_size=3, padding=1, bias=True)

        self.down1 = nn.MaxPool2d(2)
        self.conv11 = nn.Conv2d(in_channels=channel, out_channels=channel, kernel_size=3, padding=1, bias=True)
        self.up11 = Upsample(channel, channel, 2)
        self.down2 = nn.MaxPool2d(2)
        self.conv21 = nn.Conv2d(in_channels=channel, out_channels=channel, kernel_size=3, padding=1, bias=True)
        self.up21 = Upsample(channel, channel, 4)
        self.conv_out = nn.Conv2d(in_channels=3*channel, out_channels=channel, kernel_size=3, padding=1, bias=True)
    def forward(self, x):
        x00 = self.activation(self.conv00(x)) # 16
        x01 = self.activation(self.conv01(x00)) # 16
        x0_out = self.sigmoid(x01)
        x0_out = x * x0_out
        
        x11 = self.down1(x00) # 16
        x11 = self.activation(self.conv11(x11)) # 16
        x1_out = self.up11(x11) # 16
        x1_out = self.sigmoid(x1_out)
        x1_out = x * x1_out
        x21 = self.down2(x11) # 16
        x21 = self.activation(self.conv21(x21)) # 16
        x2_out = self.up21(x21) # 16
        x2_out = self.sigmoid(x2_out)
        x2_out = x * x2_out
        y = torch.cat([x0_out, x1_out, x2_out], dim=1)
        y = self.activation(self.conv_out(y)) # 16
        return y

class UniversalEstimator(nn.Module):
    def __init__(self, n_channels, n_classes, bilinear, sigmoid):
        super(UniversalEstimator, self).__init__()
        if sigmoid:
            self.estimate = nn.Sequential(
                    UNet(n_channels=n_channels, n_classes=n_classes, bilinear=bilinear),
                    PALayer(n_classes),
                    nn.Sigmoid()
            )
        else:
            self.estimate = nn.Sequential(
                    Unet(n_channels=n_channels, n_classes=n_classes, bilinear=bilinear),
                    PALayer(n_classes)
            )

    def forward(self, x):
        y = self.estimate(x)
        return y

class CALayer(nn.Module):
    def __init__(self, channel):
        super(CALayer, self).__init__()
        self.ca = nn.Sequential(
                nn.AvgPool2d(16),
                nn.Conv2d(channel, channel, kernel_size=3, padding=1, bias=True),
                nn.ReLU(inplace=True)
        )
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        y = self.ca(x)
        y = self.avg_pool(y)
        y = self.sigmoid(y)
        return x * y

# ****** Define Generator and Discriminator ******
class AtmLightEstimator(nn.Module):
	def __init__(self, in_channels, out_channels):
		super(AtmLightEstimator, self).__init__()
		self.conv = nn.Conv2d(in_channels=in_channels,out_channels=out_channels,kernel_size=3,stride=1,padding=1,bias=False)
		self.bn = nn.BatchNorm2d(num_features=in_channels)
		self.activation=nn.LeakyReLU(0.2, inplace=True)
		self.down1 = Downsample(out_channels, out_channels*2, 2)
		self.down2 = Downsample(out_channels*2, out_channels*4, 2)
		self.down3 = Downsample(out_channels*4, out_channels*8, 2)
		self.up1 = Upsample(out_channels*8, out_channels*4, 2)
		self.up2 = Upsample(out_channels*8, out_channels*4, 2)
		self.up3 = Upsample(out_channels*6, out_channels*3, 2)
		self.ca = CALayer(out_channels*3)
		self.pa = PALayer(out_channels*3)
		self.transition = TransitionLayer(out_channels*3, out_channels, False)
		self.bypass = TransitionLayer(out_channels, out_channels, False)

	def forward(self, x):
		x = self.conv(self.activation(self.bn(x)))
		feature2x = self.down1(x) # in_channels*2
		feature4x = self.down2(feature2x) #in_channels*4
		feature8x = self.down3(feature4x) #in_channels*8
		output = self.up1(feature8x) #in_channels*4
		output = self.up2(torch.cat((output, feature4x),1)) #in_channels*4
		output = self.up3(torch.cat((output, feature2x),1))
		A = self.ca(output)
		A = self.pa(A)
		A = self.transition(A) + self.bypass(x)
		# shape_out1 = A.data.size()
		# shape_out = shape_out1[2:4]
		# 
		# A = F.avg_pool2d(A, shape_out1[2])
		# A = self.upsample(self.activation(A),size=shape_out)
		return A
		
class dehaze_net(nn.Module):

	def __init__(self, input_dim=3, growthRate=12, num_dense_layers=5,reduction=0.5):
		super(dehaze_net, self).__init__()

		self.t_estimator = nn.Sequential(
                    UNet(n_channels=input_dim, n_classes=3, bilinear=False),
                    nn.Sigmoid()
            )

		self.A_estimator = nn.Sequential(
                    UNet(n_channels=input_dim, n_classes=3, bilinear=False),
                    nn.Sigmoid()
            )
		
	def forward(self, x, enable_A=1, true_A=0, enable_t=1, true_t=1):
		# if flag = 1, use true_A and abandon trained A.
		t = self.t_estimator(x)
		A = self.A_estimator(x)

		clean_image = (x-A)/t + A
		
		return clean_image, t, A

class enhance_net(nn.Module):

    def __init__(self, input_dim=3, growthRate=12, num_dense_layers=5):
        super(enhance_net, self).__init__()

        num_channels = input_dim
        self.conv1 = nn.Conv2d(num_channels, 20, 1, padding=0, bias=True)
        self.conv2 = nn.Conv2d(20, 40, 1, padding=0, bias=True)
        self.conv3 = nn.Conv2d(40, 60, 1, padding=0, bias=True)
        self.EM1 = nn.Sequential(
                    DenseBlock(60, growthRate, num_dense_layers),
                    TransitionLayer(60+growthRate*num_dense_layers, 60),
                    PALayer(60),
                    CALayer(60)
            )
        self.EM2 = nn.Sequential(
                    DenseBlock(60, growthRate, num_dense_layers),
                    TransitionLayer(60+growthRate*num_dense_layers, 60),
                    PALayer(60),
                    CALayer(60)
            )
        self.EM3 = nn.Sequential(
                    DenseBlock(60, growthRate, num_dense_layers),
                    TransitionLayer(60+growthRate*num_dense_layers, 60),
                    PALayer(60),
                    CALayer(60)
            )
        self.EM4 = nn.Sequential(
                    DenseBlock(60, growthRate, num_dense_layers),
                    TransitionLayer(60+growthRate*num_dense_layers, 60),
                    PALayer(60),
                    CALayer(60)
            )
        self.EM5 = nn.Sequential(
                    DenseBlock(60, growthRate, num_dense_layers),
                    TransitionLayer(60+growthRate*num_dense_layers, 60),
                    PALayer(60),
                    CALayer(60)
            )
        self.EM6 = nn.Sequential(
                    DenseBlock(60, growthRate, num_dense_layers),
                    TransitionLayer(60+growthRate*num_dense_layers, 60),
                    PALayer(60),
                    CALayer(60)
            )
        self.conv4 = nn.Conv2d(60, 40, 1, padding=0, bias=True)
        self.conv5 = nn.Conv2d(40, 20, 1, padding=0, bias=True)
        self.conv6 = nn.Conv2d(20, num_channels, 1, padding=0, bias=True)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        output = self.relu(self.conv1(x))
        output = self.relu(self.conv2(output))
        output = self.relu(self.conv3(output))
        
        output = self.EM1(output)
        output = self.EM2(output)
        output = self.EM3(output)
        output = self.EM4(output)
        output = self.EM5(output)
        output = self.EM6(output)
        
        output = self.relu(self.conv4(output))
        output = self.relu(self.conv5(output))
        output = self.relu(self.conv6(output))
        return output
        
ndf = 64
class Discriminator(nn.Module):
    def __init__(self, input_nc=3, ndf=64, n_layers=3, norm_layer=nn.BatchNorm2d, use_sigmoid=True):
        super(Discriminator, self).__init__()
        if type(norm_layer) == functools.partial:
            use_bias = norm_layer.func == nn.InstanceNorm2d
        else:
            use_bias = norm_layer == nn.InstanceNorm2d

        kw = 4
        padw = 1
        sequence = [
            nn.Conv2d(input_nc, ndf, kernel_size=kw, stride=2, padding=padw),
            nn.LeakyReLU(0.2, True)
        ]

        nf_mult = 1
        nf_mult_prev = 1
        for n in range(1, n_layers):
            nf_mult_prev = nf_mult
            nf_mult = min(2**n, 8)
            sequence += [
                nn.Conv2d(ndf * nf_mult_prev, ndf * nf_mult,
                          kernel_size=kw, stride=2, padding=padw, bias=use_bias),
                norm_layer(ndf * nf_mult),
                nn.LeakyReLU(0.2, True)]
        nf_mult_prev = nf_mult
        nf_mult = min(2**n_layers, 8)
        sequence += [
            nn.Conv2d(ndf * nf_mult_prev, ndf * nf_mult,
                      kernel_size=kw, stride=1, padding=padw, bias=use_bias),
            norm_layer(ndf * nf_mult),
            nn.LeakyReLU(0.2, True)
        ]

        sequence += [nn.Conv2d(ndf * nf_mult, 1, kernel_size=kw, stride=1, padding=padw)]

        if use_sigmoid:
            sequence += [nn.Sigmoid()]

        self.model = nn.Sequential(*sequence)

    def forward(self, input):
        return self.model(input)







In [4]:
import torch
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
from math import exp
from PIL import Image
import cv2
import os
import json
import math

def gaussian(window_size, sigma):
    gauss = torch.Tensor([math.exp(-(x - window_size // 2) ** 2 / float(2 * sigma ** 2)) for x in range(window_size)])
    return gauss / gauss.sum()


def create_window(window_size, channel):
    _1D_window = gaussian(window_size, 1.5).unsqueeze(1)
    _2D_window = _1D_window.mm(_1D_window.t()).float().unsqueeze(0).unsqueeze(0)
    window = Variable(_2D_window.expand(channel, 1, window_size, window_size).contiguous())
    return window


def _ssim(img1, img2, window, window_size, channel, size_average=True):
    mu1 = F.conv2d(img1, window, padding=window_size // 2, groups=channel)
    mu2 = F.conv2d(img2, window, padding=window_size // 2, groups=channel)

    mu1_sq = mu1.pow(2)
    mu2_sq = mu2.pow(2)
    mu1_mu2 = mu1 * mu2

    sigma1_sq = F.conv2d(img1 * img1, window, padding=window_size // 2, groups=channel) - mu1_sq
    sigma2_sq = F.conv2d(img2 * img2, window, padding=window_size // 2, groups=channel) - mu2_sq
    sigma12 = F.conv2d(img1 * img2, window, padding=window_size // 2, groups=channel) - mu1_mu2

    C1 = 0.01 ** 2
    C2 = 0.03 ** 2

    ssim_map = ((2 * mu1_mu2 + C1) * (2 * sigma12 + C2)) / ((mu1_sq + mu2_sq + C1) * (sigma1_sq + sigma2_sq + C2))

    if size_average:
        return ssim_map.mean()
    else:
        return ssim_map.mean(1).mean(1).mean(1)


class SSIM(torch.nn.Module):
    def __init__(self, window_size=11, size_average=True):
        super(SSIM, self).__init__()
        self.window_size = window_size
        self.size_average = size_average
        self.channel = 1
        self.window = create_window(window_size, self.channel)

    def forward(self, img1, img2):
        (_, channel, _, _) = img1.size()

        if channel == self.channel and self.window.data.type() == img1.data.type():
            window = self.window
        else:
            window = create_window(self.window_size, channel)

            if img1.is_cuda:
                window = window.cuda(img1.get_device())
            window = window.type_as(img1)

            self.window = window
            self.channel = channel

        return _ssim(img1, img2, window, self.window_size, channel, self.size_average)


def ssim(img1, img2, window_size=11, size_average=True):
    img1 = correctsize(img1)
    img2 = correctsize(img2)
    img1 = Variable(img1)
    img2 = Variable(img2)
    (_, channel, _, _) = img1.size()
    window = create_window(window_size, channel)

    if img1.is_cuda:
        window = window.cuda(img1.get_device())
    window = window.type_as(img1)

    return _ssim(img1, img2, window, window_size, channel, size_average)

def correctsize(img):
    im = torch.from_numpy(img).float().permute(2,0,1).unsqueeze(0)
    return im

In [ ]:
orig_images_path="data/images/"
hazy_images_path="data/data/"
lr=0.0001
weight_decay=0.0001
grad_clip_norm=0.1
num_epochs=2000
train_batch_size=1
val_batch_size=8
num_workers=4
display_iter=10
snapshot_iter=200
snapshots_folder="snapshots/"
sample_output_folder="samples/"

In [ ]:
import os
import sys

import torch
import torch.utils.data as data

import numpy as np
from PIL import Image
import glob
import random
import cv2
import math

random.seed(1143)


def DarkChannel(im, sz=15):
    b, g, r = cv2.split(im)
    dc = cv2.min(cv2.min(r, g), b)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (sz, sz))
    dark = cv2.erode(dc, kernel)
    return dark
def AtmLight(im, dark):
    [h, w] = im.shape[:2]
    imsz = h * w
    numpx = int(max(math.floor(imsz / 1000), 1))
    darkvec = dark.reshape(imsz, 1)
    imvec = im.reshape(imsz, 3)

    indices = darkvec.argsort()
    indices = indices[imsz - numpx::]

    atmsum = np.zeros([1, 3])
    for ind in range(1, numpx):
        atmsum = atmsum + imvec[indices[ind]]

    A = atmsum / numpx
    return A

def get_params(origin_img, hazy_img):
    J = origin_img.astype('float64') / 255
    I = hazy_img.astype('float64') / 255
    dc = DarkChannel(I)
    A = AtmLight(I, dc)
    t = (I - A[0]) / (J - A[0])
    return t, A.mean()

def populate_train_list_NYU2():


	hazy_list = tuple(["NYU2_water/haze/"+ i for i in os.listdir("NYU2_water/haze/")])
	clean_list = tuple(["NYU2/images/"+ i for i in os.listdir("NYU2/images/")])
	populated_list = []
	for i in range(1449):
		index = str(i)
		for j in range(4):
			pair = ["NYU2/images/"+str(i)+".jpg", "NYU2_water/haze/"+str(i)+"_"+str(j)+".png", "NYU2_water/A/"+str(i)+"_"+str(j)+".npy", "NYU2_water/transmission/"+str(i)+"_"+str(j)+".npy"]
			populated_list.append(pair)

	random.shuffle(populated_list)

	return populated_list

def populate_train_list_CycleGAN():


	hazy_list = tuple(["dataset_water2/"+ i for i in os.listdir("dataset_water2/") if "fake" in i])
	clean_list = tuple(["dataset_water2/"+ i for i in os.listdir("dataset_water2/") if "real" in i])
	populated_list = []
	for i in range(len(hazy_list)):
		for j in range(len(clean_list)):
			if hazy_list[i].split("_")[1]==clean_list[j].split("_")[1]:
				populated_list.append([clean_list[j], hazy_list[i], 0, 0])
				break


	random.shuffle(populated_list)

	return populated_list

	
def populate_train_list(orig_images_path, hazy_images_path):
    nyu_list = populate_train_list_NYU2()
    cyclegan_list = populate_train_list_CycleGAN()
    l = []
    p = 0
    q = 0
    for i in range(min(len(nyu_list), len(cyclegan_list))):
        if i % 4 ==0 or i % 4 ==1:
            l.append(cyclegan_list[p])
            p+=1
        else:
            l.append(nyu_list[q])
            q+=1
    return l

class dehazing_loader(data.Dataset):

	def __init__(self, orig_images_path, hazy_images_path, mode='train'):

		self.data_list = populate_train_list_CycleGAN()

		

	def __getitem__(self, index):
		netSize = 256
        
		data_orig_path, data_hazy_path, data_A_path, data_t_path = self.data_list[index]

		data_orig = Image.open(data_orig_path)
		data_hazy = Image.open(data_hazy_path)
		# est_t, est_A = get_params(np.asarray(data_orig), np.asarray(data_hazy))
		# derived = get_params(np.asarray(data_orig), np.asarray(data_hazy))
		# K = Image.fromarray(np.uint8(255*derived[0]))
		# b = Image.fromarray(np.uint8(255*derived[1]))
		# dc = DarkChannel(np.asarray(data_orig.astype('float64') / 255))

		data_orig = data_orig.resize((netSize,netSize), Image.ANTIALIAS)
		data_hazy = data_hazy.resize((netSize,netSize), Image.ANTIALIAS)

		# dc = dc.resize((480,640), Image.ANTIALIAS)
		# K = K.resize((480,640), Image.ANTIALIAS)
		# b = b.resize((480,640), Image.ANTIALIAS)

		data_orig = (np.asarray(data_orig)/255.0)
		data_hazy = (np.asarray(data_hazy)/255.0)
		# dc = (np.asarray(dc)/255.0) 
		# K = (np.asarray(K)/255.0) 
		# b = (np.asarray(data_hazy)/255.0)
		# test = K*data_hazy+b
		# if index == 0:
		# 	cv2.imshow("clear",test)
		# 	cv2.waitKey()

		data_orig = torch.from_numpy(data_orig).float()
		data_hazy = torch.from_numpy(data_hazy).float()
        
		data_A = torch.ones(1).float()
		data_t = torch.ones(1).float()
        
		if data_A_path!=0:
			data_A1 = np.load(data_A_path)
			b, g, r = cv2.split(data_A1)
			data_A = cv2.merge([r, g, b])
			data_A = cv2.resize(data_A, (netSize, netSize))
			data_A = torch.from_numpy(data_A).float().permute(2,0,1)
			data_t = np.load(data_t_path)
			b, g, r = cv2.split(data_t)
			data_t = cv2.merge([r, g, b])
			data_t = cv2.resize(data_t, (netSize, netSize))
			data_t = torch.from_numpy(data_t).float().permute(2,0,1)
		else:
			data_A = torch.Tensor([1.0]).float()
		# dc = torch.from_numpy(dc).float()
		# K = torch.from_numpy(K).float()
		# b = torch.from_numpy(b).float()

		return data_orig.permute(2,0,1), data_hazy.permute(2,0,1), data_t, data_A

	def __len__(self):
		return len(self.data_list)

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torch.backends.cudnn as cudnn
import torch.optim
import torch.nn.functional as F
import os
import sys
import argparse
import time
import numpy as np
import cv2
from torchvision import transforms
import matplotlib.pyplot as plt
import torchvision.utils as vutils
from visdom import Visdom

from torch.autograd import Variable
import torch.nn.functional as F
import json
import time

import ipdb
import matplotlib
from tqdm import tqdm
from torch.utils import data as data_
from PIL import Image

model_name = "MSEMSSDTracking6BlocksDeWater_Asynchrony_20210306"

one = torch.FloatTensor([1])
zero = torch.FloatTensor([0])
mone = one * -1
beta1 = 0.5

lambdaGAN = 0.35
device=torch.device("cuda:2")

def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

def tensor_to_img(tensor):
    # img = np.asarray(torchvision.transforms.ToPILImage()(tensor))
    # print(tensor.shape)
    img = tensor.cpu().detach().numpy().transpose((1, 2, 0))
    return img

def img_to_Tensor(img):
    assert type(img) == np.ndarray,'the img type is {}, but ndarry expected'.format(type(img))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = torch.from_numpy(img.transpose((2, 0, 1)))
    return img.float().div(255).unsqueeze(0)
	
def get_x_gradient(tensor):
    img = tensor_to_img(tensor)
    gradient = cv2.Sobel(img,cv2.CV_64F,1,0,ksize=-1)
    return img_to_Tensor(gradient)
	
def get_y_gradient(tensor):
    img = tensor_to_img(tensor)
    gradient = cv2.Sobel(img,cv2.CV_64F,0,1,ksize=-1)
    return img_to_Tensor(gradient)

def nmz(tensor):
    shape = tensor.shape[2:]
    r_mean = torch.randn(shape)
    r_mean = r_mean.fill_(0.485).unsqueeze(0).to(device)
    g_mean = torch.randn(shape)
    g_mean = g_mean.fill_(0.456).unsqueeze(0).to(device)
    b_mean = torch.randn(shape)
    b_mean = b_mean.fill_(0.406).unsqueeze(0).to(device)
    rgb_mean=torch.cat([r_mean, g_mean, b_mean], 0)
    r_std = torch.randn(shape)
    r_std = r_std.fill_(0.229).unsqueeze(0).to(device)
    g_std = torch.randn(shape)
    g_std = g_std.fill_(0.224).unsqueeze(0).to(device)
    b_std = torch.randn(shape)
    b_std = b_std.fill_(0.225).unsqueeze(0).to(device)
    rgb_std=torch.cat([r_std, g_std, b_std], 0).unsqueeze(0)
    tensor = (tensor-rgb_mean) / rgb_std
    return tensor


def train():
    # Define Faster RCNN Constant
    SSD_.device=device
    dataset = SSD_.VOCDetection("VOCdevkit", [('2007', 'trainval')], SSD_.SSDAugmentation(300, (104, 117, 123)), SSD_.VOCAnnotationTransform())
    dataloader = data_.DataLoader(dataset, \
                                  batch_size=1, \
                                  shuffle=True, \
                                  # pin_memory=True,
                                  num_workers=4)
    SSD_net = SSD_.build_ssd('train', 300, 21)
    SSD_net = SSD_net.to(device)
    SSD_net.load_state_dict(torch.load("SSD/ssd300_mAP_77.43_v2.pth"))
    best_map = 0

    for p in SSD_net.parameters():
        p.requires_grad = False
    
    # Define Enhance Net Constant
    criterionMSE = nn.MSELoss()
    criterionSSIM = SSIM()
    criterionTracking = SSD_.MultiBoxLoss(21, 0.5, True, True, 3, 0.5, False, True)
    gamma = 0.001
    
    netEnhance = enhance_net()
    netEnhance = netEnhance.to(device)
    netEnhance.load_state_dict(torch.load('snapshots/MSEMSSDTracking6BlocksDeWater_Asynchrony_20210306_G_240.pth'))

    iteration_length = len(dataloader)
    optimizer_G = torch.optim.Adam(netEnhance.parameters(), lr=lr, betas=(beta1, 0.999))
    scheduler = lr_scheduler.CosineAnnealingLR(optimizer_G, 50)
    
    img_losses = []
    tracking_losses = []
    with open(model_name+'_img_loss.txt', 'r') as f:
        img_losses = json.loads(f.read())[:241]
    with open(model_name+'_tracking_loss.txt', 'r') as f:
        tracking_losses = json.loads(f.read())[:241]
    
    

    for epoch in range(241, num_epochs+1):
        # For each batch in the dataloader
        img_avg_loss = 0
        tracking_avg_loss = 0
        start_time = time.time()
        for iteration, (img_orig, img_haze, targets) in tqdm(enumerate(dataloader)):

            img_orig = img_orig.to(device)
            img_haze = img_haze.to(device)
            
            for flag in range(2):
                if flag ==0:
                    netEnhance.zero_grad()
                    enhanced = netEnhance(img_haze)
                    ssd_input = SSD_.ToSSDInput(enhanced)
                    ssd_output = SSD_net(enhanced)
                    img_loss = criterionMSE(enhanced, img_orig)
                    loss_l, loss_c = criterionTracking(ssd_output, targets)
                    tracking_loss = gamma * (loss_l + loss_c)
                    tracking_avg_loss += tracking_loss.item() / iteration_length
                    tracking_loss.backward()
                    optimizer_G.step()
                else:
                    netEnhance.zero_grad()
                    enhanced = netEnhance(img_haze)
                    img_loss = 1 - criterionSSIM(enhanced, img_orig)
                    img_avg_loss += img_loss.item() / iteration_length
            
                    img_loss.backward()

                    optimizer_G.step()
                    
            
        end_time = time.time()
        used_time = end_time - start_time
        print('[%d/%d]\t Image Loss: %10f | trackingLoss: %10f | used_time: %10f'
            % (epoch, num_epochs, img_avg_loss, tracking_avg_loss, used_time))
        img_losses.append(img_avg_loss)
        tracking_losses.append(tracking_avg_loss)
        if(epoch%20==0):
            torch.save(netEnhance.state_dict(), snapshots_folder + model_name + "_G_"+str(epoch)+".pth")
        with open(model_name+'_img_loss.txt', 'w') as f:
            f.write(json.dumps(img_losses))
            plt.cla()
            plt.plot(list(range(len(img_losses))), img_losses, label="blue", color="#ff0000")
            plt.savefig(model_name + '_img_loss' + '.png')
        with open(model_name+'_tracking_loss.txt', 'w') as f:
            f.write(json.dumps(tracking_losses))
            plt.cla()
            plt.plot(list(range(len(tracking_losses))), tracking_losses, label="blue", color="#ff0000")
            plt.savefig(model_name + '_tracking_loss' + '.png')
















	

train()

In [ ]:
train()

0it [00:00, ?it/s]/usr/local/lib/python3.5/dist-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
5011it [1:03:17,  1.32it/s]


[241/2000]	 Image Loss:   0.091097 | trackingLoss:   0.014891 | used_time: 3797.283684


5011it [1:03:28,  1.32it/s]


[242/2000]	 Image Loss:   0.091320 | trackingLoss:   0.014892 | used_time: 3808.690164


5011it [1:03:27,  1.32it/s]


...

5011it [1:02:53,  1.33it/s]


[593/2000]	 Image Loss:   0.102907 | trackingLoss:   0.014945 | used_time: 3773.828632


5011it [1:02:51,  1.33it/s]


[594/2000]	 Image Loss:   0.102317 | trackingLoss:   0.014944 | used_time: 3771.219462


3259it [40:49,  1.24it/s]